# LAB 4 : Collaborative Filtering on Last.fm Dataset

import library and dataset

In [8]:
import pandas as pd
import sys, os 
import numpy as np
from scipy.sparse import csr_matrix

parent_dir = os.getcwd() 
path = os.path.dirname(parent_dir)
sys.path.append(path)
# print(path)
file = path + r'\LAB4\usersha1-artmbid-artname-plays.tsv'
# print(file)
user_artist_play_df = pd.read_table(file,usecols=[0, 2, 3],names=['user', 'artist', 'plays'])
user_artist_play_df

,user,artist,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
...,...,...,...
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10


Checking and dropping missing values

In [11]:
print(user_artist_play_df.isnull().sum())
user_artist_play_df.dropna(inplace=True)

user        0
artist    204
plays       0
dtype: int64


Create sparse matrix

In [12]:
user_artist_play_df['user']     = user_artist_play_df['user'].astype("category")
user_artist_play_df['artist']   = user_artist_play_df['artist'].astype("category")

plays = csr_matrix((user_artist_play_df['plays'].astype(float), 
                   (user_artist_play_df['user'].cat.codes, 
                    user_artist_play_df['artist'].cat.codes)))
print(plays)

  (0, 19356)	229.0
  (0, 19592)	288.0
  (0, 37515)	310.0
  (0, 45554)	135.0
  (0, 45561)	2137.0
  (0, 46244)	134.0
  (0, 51085)	185.0
  (0, 90933)	1099.0
  (0, 92172)	154.0
  (0, 100639)	302.0
  (0, 103711)	232.0
  (0, 106689)	222.0
  (0, 106704)	717.0
  (0, 125484)	134.0
  (0, 126690)	361.0
  (0, 129383)	151.0
  (0, 137062)	358.0
  (0, 144238)	227.0
  (0, 144293)	316.0
  (0, 151766)	182.0
  (0, 154684)	198.0
  (0, 155241)	706.0
  (0, 165120)	135.0
  (0, 167984)	281.0
  (0, 169906)	387.0
  :	:
  (358867, 160643)	14.0
  (358867, 161415)	27.0
  (358867, 161724)	14.0
  (358867, 162097)	32.0
  (358867, 167143)	131.0
  (358867, 171784)	11.0
  (358867, 178275)	12.0
  (358867, 186549)	18.0
  (358867, 217883)	12.0
  (358867, 218091)	23.0
  (358867, 235118)	10.0
  (358867, 251158)	16.0
  (358867, 254306)	36.0
  (358867, 254532)	12.0
  (358867, 258344)	14.0
  (358867, 262827)	10.0
  (358867, 263239)	12.0
  (358867, 263330)	30.0
  (358867, 263682)	14.0
  (358867, 263831)	24.0
  (358867, 265438)	1

Weight matrix before training a model

In [30]:
from implicit.nearest_neighbours import bm25_weight
plays = bm25_weight(plays, K1=100, B=0.8)
print(plays)

  (0, 19356)	622.7789533975007
  (0, 19592)	678.9313535275278
  (0, 37515)	554.6438447819392
  (0, 45554)	765.8115724263689
  (0, 45561)	726.9229525602759
  (0, 46244)	575.8156104217842
  (0, 51085)	400.0166060949055
  (0, 90933)	274.3369272569963
  (0, 92172)	556.9355881401191
  (0, 100639)	273.0713587321172
  (0, 103711)	342.39461698803535
  (0, 106689)	349.3865898955976
  (0, 106704)	451.95682330717864
  (0, 125484)	612.543985239867
  (0, 126690)	222.88904543822787
  (0, 129383)	290.1138009884859
  (0, 137062)	430.6255867284649
  (0, 144238)	166.40720335318375
  (0, 144293)	468.41608103405565
  (0, 151766)	233.73638461351786
  (0, 154684)	251.82508898764354
  (0, 155241)	563.5375900845022
  (0, 165120)	500.7959907008511
  (0, 167984)	352.0815066178312
  (0, 169906)	472.35961084873185
  :	:
  (358867, 160643)	553.6988566325857
  (358867, 161415)	169.65521664853463
  (358867, 161724)	987.7310274172253
  (358867, 162097)	282.514042111303
  (358867, 167143)	390.5051407944552
  (358867, 

Train an ALS model using implicit

In [32]:
from implicit.als import AlternatingLeastSquares
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
plays = plays.tocsr()
model.fit(plays)

100%|██████████| 15/15 [08:18<00:00, 33.20s/it]


The result

In [33]:
userid = 8500
artists =user_artist_play_df['artist']
ids, scores = model.recommend(userid, plays[userid], N=10, filter_already_liked_items=False)
# print(ids)
pd.DataFrame({"artist": artists[ids], "score": scores, "already_liked": np.in1d(ids, plays[userid].indices)})

,artist,score,already_liked
166326,cease upon the capitol,1.079479,True
86060,the thrillseekers,1.076472,False
163964,dead infection,1.071970,True
269424,feist,1.069601,True
238158,justin timberlake,1.065312,True
136881,orishas,1.045608,True
232202,the format,1.039976,False
272422,manchester orchestra,1.039907,True
273667,madrugada,1.036065,True
199241,incinerate,1.033210,True


In [34]:
itemid = 98603	
print(f"Artist ID {itemid} : {user_artist_play_df['artist'][itemid]}")
ids, scores= model.similar_items(itemid)

# display the results using pandas for nicer formatting
pd.DataFrame({"artist": artists[ids], "score": scores})

Artist ID 98603 : the beatles


,artist,score
98603,the beatles,1.000000
214835,menomena,0.715191
172065,boz scaggs,0.711623
230159,showtek,0.696861
260432,frank zappa,0.695109
222350,lamb,0.693568
87465,signum,0.687895
85241,the prodigy,0.687232
70517,tafuri,0.685222
290559,the clash,0.682942
